In [10]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep
# 隨機休息
from random import randint
# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess


# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

# 放置爬取的資料
listData = []

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/home/ubuntuosmnx/.wdm/drivers/chromedriver/linux64/103.0.5060.53/chromedriver] found in cache


In [11]:
def visit():
    # 爬蟲網址
    driver.get('https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/');
    
def search(qString):
    # 輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, "input#ysearchinput0")
    txtInput.send_keys(qString)
    # 等待一下
    sleep(2)
    # 送出表單資料
    txtInput.submit()
    
# 分析頁面元素資訊
def get_pages():
    # 第一次取得文章列表
    ndltd_renderers = driver.find_elements(
        By.CSS_SELECTOR, 
        'td.std2 > a'
    )
    ndltd_renderers[0].click()
    # 取得論文數量
    pages = driver.find_elements(
        By.CSS_SELECTOR,
        'div.search_bar > table > tbody > tr > td'
    )[0].find_elements(
        By.CSS_SELECTOR,
        'table > tbody > tr > td'
    )[0].text
    return int(pages.split(' ')[5])

def get_content():
    # 取得論文名稱
    title = driver.find_elements(
        By.CSS_SELECTOR,
        'table.tableoutfmt2 > tbody > tr'
    )[2].text
    
    # 因研究生是否由外文名導致格式有異，故以此方式篩選
    if title[0:5] != '論文名稱:':
        title = driver.find_elements(
            By.CSS_SELECTOR,
            'table.tableoutfmt2 > tbody > tr'
        )[3].text
    sleep(1)
    
    # 變換標籤至摘要
    changeSelected = driver.find_elements(
        By.CSS_SELECTOR,
        'ul.yui-nav > li'
    )[1].click()
    sleep(1)
    
    # 取得摘要內容
    summary = driver.find_elements(
        By.CSS_SELECTOR,
        'div.yui-content > div'
    )[1].find_element(
        By.CSS_SELECTOR,
        'table > tbody > tr > td.stdncl2'
    ).text
    sleep(1)
    return title, summary

def next_page():
    # 變換至下一篇論文頁面
    n_pages = driver.find_elements(
        By.CSS_SELECTOR,
        'div.search_bar > table > tbody > tr > td'
    )[0].find_elements(
        By.CSS_SELECTOR,
        'table > tbody > tr > td'
    )[3].click()

In [13]:
# 主程式
if __name__ == '__main__':
    visit()
    search('big data')
    pages = get_pages()
    for i in range(pages):
        try:
            title, summary = get_content()
            sleep(randint(1,2))
            listData.append({
                'title': title,
                'summary': summary
            })
            print(f'{title}已收納進陣列')
        except:
            print('無法取得資料')
        next_page()

論文名稱: 政府福利政策之服務品質大數據分析：嘉義縣老人假牙補助為例已收納進陣列
論文名稱: 女性保養品品牌的大數據網路輿情分析已收納進陣列
論文名稱: 中國大陸手機品牌的大數據網路輿情分析已收納進陣列
論文名稱: 基於Google Data Studio報表設計框架之大數據分析系統設計已收納進陣列
論文名稱: 基於Dagster資料整合平台之大數據倉儲系統設計已收納進陣列
論文名稱: 元宇宙對臺灣股市之短期動能大數據分析已收納進陣列
論文名稱: 應用大數據分析於房地產價格算已收納進陣列
論文名稱: 利用大數據分析健康產業發展: 幹細胞療法的知識產權實證分析已收納進陣列
論文名稱: 探討購物網站大數據資訊介面設計對使用者經驗的影響已收納進陣列
論文名稱: 針對大數據排序之分散式FPGA運算架構與資料壓縮技術之研究已收納進陣列
無法取得資料
論文名稱: 大數據分析健檢資料庫探討老年人慢性腎臟病的影響相關因素已收納進陣列
論文名稱: 利用雲端技術進行營建行動裝置查驗及大數據統計之研究已收納進陣列
無法取得資料
論文名稱: 以鉅量資料取徑分析台灣百萬YouTuber之經營模式已收納進陣列
論文名稱: 以框架式為基礎之大數據視覺化平台架構設計已收納進陣列
論文名稱: 不同語言的網站使用者導航偏好之大數據分析已收納進陣列
論文名稱: 科技巨頭大數據蒐集與處理個人資料之研究—以2019年法國谷歌案為中心已收納進陣列
論文名稱: 智慧資本與大數據分析能力對企業內外部整合與營運績效之影響已收納進陣列
論文名稱: 大數據時代下的國際化成長策略-以 P 公司為例已收納進陣列
論文名稱: 應用機器學習與大數據分析對電子商務平台客戶忠誠度 進行統計建模與仿真分析已收納進陣列
論文名稱: 新型冠狀肺炎(COVID-19)疫苗接種之大數據與視覺化分析已收納進陣列
論文名稱: 2020年新竹市長林智堅施政的巨量分析:以Sol-Idea網路輿情大數據平台為測量工具已收納進陣列
論文名稱: 利用大數據分析在混合式學習下影響大學生數學成就之因素初探已收納進陣列
論文名稱: 大數據分析與LSTM模型於股市投資策略之研究－以台灣各類股為例已收納進陣列
論文名稱: 大數據分析應用於交通行動服務高雄MeNGo月票之顧客分群研究已收納進陣列
論文名稱: 應用大數據與分群方法發展最佳化儲位規劃與合併揀貨策略-以H

IndexError: list index out of range

In [7]:
print(listData)

[{'title': '論文名稱: 政府福利政策之服務品質大數據分析：嘉義縣老人假牙補助為例', 'summary': '本研究在了解110年接受嘉義縣補助假牙長者配戴後滿意度情形，以及對於政府福利政策品質分析。\n以隨機抽樣方式進行電話問卷，訪問內容包含：受訪者基本資料、健康狀況、生活、飲食習慣、口腔健康與保健、就醫行為與溝通、假牙裝置經驗、動機與管道、裝置調適與保養、裝置後感受與滿意度。資料蒐集以SPSS/WIN 19.0軟體進行資料建檔及統計分析，統計方法包括描述性分析(如平均值、百分比)與決策樹分析。\n研究樣本3984人，男生243人，女生246人接受電話訪問，補助年齡層以66-70歲最多，假牙裝置態樣以全口活動假牙最多(40.8%)，多數自覺咀嚼食物能力不好佔(81.4%)，以往有使用假牙經驗(52.8%)，接受假牙裝置後，咬合不舒適疼痛為最多(50.7%)、其次為鬆動(21.6%)，曾經有使用假牙經驗者(52.8%)逾半數以上，使用時間以3~5年為多數，目前有2副以上假牙者僅佔(5.1%)，影響裝置滿意度因素為「嚼食能力感受」，在照護政策滿意度分析，女性、口腔不健康、經濟狀況足夠應付者較滿意度較低，整體裝置滿意度為(50.9%)，照護服務整體滿意度為(51.3%)，政策滿意度為(98%)。\n嘉義縣假牙補助政策獲得民眾肯定認同，長者假牙裝置後嚼食能力獲得改善，對飲食攝取有幫助，初戴假牙患者常見咬合不適應，公務部門可持續追蹤，鼓勵回醫療院所調整，並結合社區活動或針對完成裝置長者，推廣口腔保健知識與假牙清潔維護技能，提升長者口腔健康及生活品質。'}, {'title': '論文名稱: 女性保養品品牌的大數據網路輿情分析', 'summary': '隨著新媒體蓬勃發展，網紅業配以及明星代言，引領著流行趨勢。除此之外，美妝博主也會透過社群網路平台，例如：Facebook、Youtube以及Instagram等，分享保養的方式。消費者可以透過隨手可得的行動裝置輕易取得資訊，直接接觸到相關資訊，導致消費者對於自身皮膚的保養意識覺醒。保養品涵括了化妝水、精華液、面膜、眼部精華、眼霜、乳液和防曬等等。\n疫情期間，大部分民眾若非必要基本上都足不出戶。若民眾不得已需要出門，基於防疫需求，需要配戴口罩。民眾的臉蛋長期被悶住，容易產生皮膚敏感、乾燥、粉刺爆增等狀況。因此針對